### Semantic Search Setup 

In [4]:
!pip install sentence-transformers --quiet

In [6]:
import singlestoredb as s2
import sqlalchemy as sa
import numpy as np
from sentence_transformers import SentenceTransformer, util

In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
documents = [
    "The crispy golden fries complement the juicy burger perfectly.",
    "I had a delightful dinner at the bistro with my family.",
    "The menu is showing signs of innovation after the chef change.",
    "The surface of the crème brûlée is caramelized to perfection.",
    "Fusion cuisine techniques have become very sophisticated.",
    "The ambiance of the restaurant creates a cozy and inviting atmosphere.",
    "The seafood paella was bursting with flavor and freshness.",
    "Service was attentive and friendly, making our dining experience enjoyable.",
    "The dessert menu features an exquisite chocolate lava cake.",
    "I highly recommend the house special, which is a delightful twist on classic Italian.",
    "The wine selection pairs beautifully with the diverse menu options.",
    "The vegetarian options are creative and satisfying for all tastes.",
    "Our server provided excellent recommendations that enhanced our meal.",
    "The brunch menu includes a variety of delicious pancakes and omelets.",
    "Every bite of the steak was tender and cooked to perfection.",
    "The spicy curry left a lasting impression with its rich flavors.",
    "I appreciated the locally sourced ingredients used in each dish.",
    "The outdoor seating area offers a lovely view of the sunset.",
    "Happy hour specials made for a great value on drinks and appetizers.",
    "The chef's tasting menu is a must-try for adventurous diners.",
    "Overall, it was an unforgettable dining experience that I can't wait to repeat."
]

In [9]:
document_embeddings = model.encode(documents)

In [10]:
lengths = [len(document_embeddings_items) for document_embeddings_items in document_embeddings]

print("Lengths of outer elements:", lengths)

Lengths of outer elements: [384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384]


Dimensions above and below

In [17]:
%%sql
CREATE TABLE reviews(id INT AUTO_INCREMENT PRIMARY KEY, reviews TEXT, reviews_embeddings VECTOR(384));

++
||
++
++

In [23]:
%config SqlMagic.named_parameters="enabled"

In [18]:
conn = s2.connect()

with conn.cursor() as cur:
    for x in range(len(documents)):
        cur.execute("INSERT INTO reviews (reviews, reviews_embeddings) VALUES (%s, %s)", (documents[x], document_embeddings[x]))

### WITH A LESS SPECIFIC QUERY 

In [19]:
query = "Give me fries"

In [20]:
query_embedding = model.encode(query)

In [21]:
query_embedding = str(query_embedding.tolist())

In [26]:
%%sql
SET @target_query = :query_embedding;

SELECT id, reviews, reviews_embeddings <*> @target_query as similarity FROM reviews
ORDER BY reviews_embeddings <*> @target_query DESC
LIMIT 10;

10 rows affected.

id,reviews,similarity
2251799813685249,The crispy golden fries complement the juicy burger perfectly.,0.39128178358078003
2251799813685262,The brunch menu includes a variety of delicious pancakes and omelets.,0.27463364601135254
2251799813685268,The chef's tasting menu is a must-try for adventurous diners.,0.22095204889774323
2251799813685261,Our server provided excellent recommendations that enhanced our meal.,0.2021971493959427
2251799813685250,I had a delightful dinner at the bistro with my family.,0.17907264828681946
2251799813685264,The spicy curry left a lasting impression with its rich flavors.,0.16737140715122223
2251799813685251,The menu is showing signs of innovation after the chef change.,0.15721024572849274
2251799813685260,The vegetarian options are creative and satisfying for all tastes.,0.15684184432029724
2251799813685263,Every bite of the steak was tender and cooked to perfection.,0.14731888473033905
2251799813685256,"Service was attentive and friendly, making our dining experience enjoyable.",0.14573320746421814


### WITH A MORE SPECIFIC QUERY 

In [27]:
query = "I would like fries and a good ambiance with a friendly chef"

In [28]:
query_embedding = model.encode(query)

In [29]:
query_embedding = str(query_embedding.tolist())

In [30]:
%%sql
SET @target_query = :query_embedding;

SELECT id, reviews, reviews_embeddings <*> @target_query as similarity FROM reviews
ORDER BY reviews_embeddings <*> @target_query DESC
LIMIT 10;

10 rows affected.

id,reviews,similarity
2251799813685268,The chef's tasting menu is a must-try for adventurous diners.,0.5724650025367737
2251799813685249,The crispy golden fries complement the juicy burger perfectly.,0.45793285965919495
2251799813685261,Our server provided excellent recommendations that enhanced our meal.,0.4116223454475403
2251799813685254,The ambiance of the restaurant creates a cozy and inviting atmosphere.,0.39965468645095825
2251799813685256,"Service was attentive and friendly, making our dining experience enjoyable.",0.3764159679412842
2251799813685250,I had a delightful dinner at the bistro with my family.,0.3706723749637604
2251799813685251,The menu is showing signs of innovation after the chef change.,0.35536593198776245
2251799813685267,Happy hour specials made for a great value on drinks and appetizers.,0.34272971749305725
2251799813685262,The brunch menu includes a variety of delicious pancakes and omelets.,0.3385392427444458
2251799813685269,"Overall, it was an unforgettable dining experience that I can't wait to repeat.",0.328262060880661
